# Combine Data Sets
## groupby 활용하여 다양한 데이터 집계를 활용하기
- 두 개의 데이터프레임을 위아래로 합치는 것이아니라(pd.concat([df1, df2])
- 두 개의 데이터프레임을 `옆으로` 합치는 것 `pd.merge()`
<br><br>
이러한 merge()는 주로 컬럼이 여러개가 있을대, 그 컬럼끼리 값을 합칠때 사용하면 좋다. <br>
<br>
<b>concat은 위아래로 합칠때, merge는 옆으로 합칠때 주로 사용한다.</b>

In [1]:
import pandas as pd

In [2]:
# Pandas Cheat Sheet의 데이터프레임을 그대로 만들어보자
# adf라는 이름의 데이터프레임을 만들어주자
adf = pd.DataFrame({"x1" : ["A", "B", "C"],
                    "x2" : [1, 2, 3]})

adf

,x1,x2
0,A,1
1,B,2
2,C,3


In [3]:
# bdf라는 이름의 데이터프레임을 만들어주자
bdf = pd.DataFrame({"x1" : ["A", "B", "D"],
                    "x3" : ["T", "F", "T"]})
bdf

,x1,x3
0,A,T
1,B,F
2,D,T


In [6]:
# adf와 bdf를 조인한다.
# 이때, left를 기준으로하며, x1을 key로해서 머지한다.
pd.merge(adf, bdf, how='left', on='x1')

,x1,x2,x3
0,A,1,T
1,B,2,F
2,C,3,NaN


adf가 left, bdf가 right가 된다. <br>
그래서 left인 adf를 기준으로 merge가 된다. 따라서 adf에 있는 A, B, C를 가져온다. <br>
이때 bdf에는 A, B만 있기에 A, B만 조인한다. C가 없으니 NaN으로 조인된다.

In [7]:
pd.merge(adf, bdf, how='right', on='x1')

,x1,x2,x3
0,A,1.0,T
1,B,2.0,F
2,D,NaN,T


=> 여기선 right인 bdf가 기준이되어 merge 되었다. key값은 x1 
<br>
adf의 C가 있는 행은 조인되지 않는다. 

In [9]:
pd.merge(adf, bdf, how = 'inner', on = 'x1')

,x1,x2,x3
0,A,1,T
1,B,2,F


=> 여기선 둘다 해당되는 값만 merge된다.

In [10]:
pd.merge(adf, bdf, how='outer', on='x1')

,x1,x2,x3
0,A,1.0,T
1,B,2.0,F
2,C,3.0,NaN
3,D,NaN,T


=> 여기서는 비어있는 모든 값들을 가져오면서 비어있는 값들에는 NaN이 들어간다.

### 이때 Filtering Joins이라 하여 특정 값들을 가져와 조인할 수도 있다.
- 값을 필터링하여 조인한다

### adf[adf.x1.isin(bdf.x1)]

In [12]:
bdf.x1

0    A
1    B
2    D
Name: x1, dtype: object

In [13]:
# bdf.x1이 adf.x1에도 있으면 True
adf.x1.isin(bdf.x1)

0     True
1     True
2    False
Name: x1, dtype: bool

In [14]:
# 위를 다시 adf로 감싸면 adf에 있는 값만 가지고 올 수 있다.
adf[adf.x1.isin(bdf.x1)]

,x1,x2
0,A,1
1,B,2


### adf[~adf.x1.isin(bdf.x1)]
- ~은 제외함을 의미한다. 

In [15]:
# 위의 값은 제외하고 C값만 가져왔다.
adf[~adf.x1.isin(bdf.x1)]

,x1,x2
2,C,3


## ydf, zdf 데이터프레임만들기
- 컬럼값은 같은 이 두개의 데이터프레임을 어떻게 합쳐야 할까?
- 컬럼의 값이 같으니 그냥 merge 해주자.

In [16]:
ydf = pd.DataFrame({"x1" : ["A", "B", "C"],
                    "x2" : [1, 2, 3]})

ydf

,x1,x2
0,A,1
1,B,2
2,C,3


In [17]:
zdf = pd.DataFrame({"x1" : ["B", "C", "D"],
                    "x2" : [2, 3, 4]})

zdf

,x1,x2
0,B,2
1,C,3
2,D,4


In [18]:
# 기본 값은 inner join이므로 두개의 공통값이 있는 것만 조인된다. 
pd.merge(ydf, zdf)

,x1,x2
0,B,2
1,C,3


In [19]:
# how = 'outer'를 통해 공통값이 아닌 것도 merge된다
pd.merge(ydf, zdf, how = 'outer')

,x1,x2
0,A,1
1,B,2
2,C,3
3,D,4


<b>pd.merge(ydf, zdf, how='outer', indicator=True).query('__merge == "left___only"').drop(columns=['_merge'])</b>

In [21]:
# _merge라는 컬럼이 새로 생김
# 얘가 어떻게 merge를 한건지 보여준다. 
# __merge라는 컬럼을 만들고 indicator라는 옵션을 주면 이러한 값들이 생성된다.
pd.merge(ydf, zdf, how = 'outer', indicator = True)

,x1,x2,_merge
0,A,1,left_only
1,B,2,both
2,C,3,both
3,D,4,right_only


In [25]:
# 위의 상태에서 left_only값만 가져오기
pd.merge(ydf, zdf, how='outer', indicator=True).query('_merge == "left_only"')

,x1,x2,_merge
0,A,1,left_only


In [27]:
# 위의 상태에서 _merge컬럼을 drop시킨다
pd.merge(ydf, zdf, how='outer', indicator=True).query('_merge == "left_only"').drop(columns=['_merge'])

,x1,x2
0,A,1


In [28]:
pd.merge?